# Notebook to build an enriched case log from an event log

In [1]:
import pandas as pd
from datetime import datetime

## 0. Data loading

In [2]:
INPUT_FILE_NAME = 'rtfm_full.csv' # a csv version of https://doi.org/10.4121/uuid:270fd440-1057-4fb9-89a9-b699b47990f5
CASE_ID_COL = 'case:concept:name'
TIMESTAMP_COL = 'time:timestamp'
ACTIVITY_COL = 'concept:name'

In [3]:
df = pd.read_csv(INPUT_FILE_NAME,  dtype={'org:resource': str, 'matricola': str}, parse_dates=[TIMESTAMP_COL])
print(len(df), 'rows read from', INPUT_FILE_NAME)
df = df.sort_values(TIMESTAMP_COL, ignore_index=True)
df.info()
df.head()

561470 rows read from rtfm_full.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561470 entries, 0 to 561469
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   amount                230230 non-null  float64       
 1   org:resource          150925 non-null  object        
 2   dismissal             155066 non-null  object        
 3   concept:name          561470 non-null  object        
 4   vehicleClass          150370 non-null  object        
 5   totalPaymentAmount    227971 non-null  float64       
 6   lifecycle:transition  561470 non-null  object        
 7   time:timestamp        561470 non-null  datetime64[ns]
 8   article               150370 non-null  float64       
 9   points                150370 non-null  float64       
 10  case:concept:name     561470 non-null  object        
 11  expense               103987 non-null  float64       
 12  notificationType      

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
0,62.59,18,NIL,Create Fine,A,0.0,complete,2000-01-01,158.0,0.0,S38735,NaN,NaN,NaN,NaN,NaN
1,62.59,704,NIL,Create Fine,A,0.0,complete,2000-01-02,158.0,0.0,S44306,NaN,NaN,NaN,NaN,NaN
2,31.30,35,NIL,Create Fine,A,0.0,complete,2000-01-02,158.0,0.0,S49055,NaN,NaN,NaN,NaN,NaN
3,31.30,35,NIL,Create Fine,A,0.0,complete,2000-01-02,158.0,0.0,S49056,NaN,NaN,NaN,NaN,NaN
4,31.30,35,NIL,Create Fine,A,0.0,complete,2000-01-02,158.0,0.0,S49057,NaN,NaN,NaN,NaN,NaN


## 1. Event enrichment

In [4]:
HIGHFLOAT = 2000000.0
# 1. Case History Aggregation: keep track of current state of a variable trough history of a case
TRACK = [
    ('amount', 'last', 0.0),
    ('amount', 'min', HIGHFLOAT),
    # ('totalPaymentAmount', 'max', 0.0), # uncomment to check the issues with totalPaymentAmount
    ('paymentAmount', 'sum', 0.0),
    ('dismissal', 'last', 'missing'),
    ('expense', 'sum', 0.0)
    ]
# 2. Add variables derived from other values of a row (i.e. event)
COMPUTE = [
    ('outstanding_balance', lambda x: round(x['amount::last'] + x['expense::sum'] - x['paymentAmount::sum'], 2)),
    ('outstanding_balance_without_expense', lambda x: round(x['amount::last'] - x['paymentAmount::sum'], 2)),
    ('outstanding_balance_without_penalty', lambda x: round(x['amount::min'] + x['expense::sum'] - x['paymentAmount::sum'], 2))
]

value_store = {} # map each tracked variable (incl method) and case to its current value
added_cols = {} # map each tracked variable (incl method) to a growing list/vector of values
added_col_names = []

for (col, method, _) in TRACK:
    added_cols[(col, method)] = []

for index, row in df.iterrows():
    case_id = row[CASE_ID_COL]
    for (col, method, init_val) in TRACK:
        if not (case_id, col, method) in value_store:
            value_store[(case_id, col, method)] = init_val
        if not pd.isna(row[col]):
            if method == 'last':
                value_store[(case_id, col, method)] = row[col]
            elif method == 'max':
                value_store[(case_id, col, method)] = max(row[col],value_store[(case_id, col, method)])
            elif method == 'min':
                value_store[(case_id, col, method)] = min(row[col],value_store[(case_id, col, method)])
            elif method == 'sum':
                value_store[(case_id, col, method)] = round(value_store[(case_id, col, method)] + row[col], 2)
            else:
                print('NOT SUPPORTED')
        added_cols[(col, method)].append(value_store[(case_id, col, method)])

for (col, method, _) in TRACK:
    tracking_col_name = col+'::'+method
    added_col_names.append(tracking_col_name)
    df[tracking_col_name] = added_cols[(col,method)]
        

for (name, function) in COMPUTE:
    added_col_names.append(name)
    df[name] = df.apply(function, axis=1)


In [5]:
intermediate_file_name = 'X' + INPUT_FILE_NAME
df.to_csv(intermediate_file_name, index=False)
print(len(df), 'rows written to', intermediate_file_name)
df.info()
df.head()

561470 rows written to Xrtfm_full.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561470 entries, 0 to 561469
Data columns (total 24 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   amount                               230230 non-null  float64       
 1   org:resource                         150925 non-null  object        
 2   dismissal                            155066 non-null  object        
 3   concept:name                         561470 non-null  object        
 4   vehicleClass                         150370 non-null  object        
 5   totalPaymentAmount                   227971 non-null  float64       
 6   lifecycle:transition                 561470 non-null  object        
 7   time:timestamp                       561470 non-null  datetime64[ns]
 8   article                              150370 non-null  float64       
 9   points                          

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,...,paymentAmount,matricola,amount::last,amount::min,paymentAmount::sum,dismissal::last,expense::sum,outstanding_balance,outstanding_balance_without_expense,outstanding_balance_without_penalty
0,62.59,18,NIL,Create Fine,A,0.0,complete,2000-01-01,158.0,0.0,...,NaN,NaN,62.59,62.59,0.0,NIL,0.0,62.59,62.59,62.59
1,62.59,704,NIL,Create Fine,A,0.0,complete,2000-01-02,158.0,0.0,...,NaN,NaN,62.59,62.59,0.0,NIL,0.0,62.59,62.59,62.59
2,31.30,35,NIL,Create Fine,A,0.0,complete,2000-01-02,158.0,0.0,...,NaN,NaN,31.30,31.30,0.0,NIL,0.0,31.30,31.30,31.30
3,31.30,35,NIL,Create Fine,A,0.0,complete,2000-01-02,158.0,0.0,...,NaN,NaN,31.30,31.30,0.0,NIL,0.0,31.30,31.30,31.30
4,31.30,35,NIL,Create Fine,A,0.0,complete,2000-01-02,158.0,0.0,...,NaN,NaN,31.30,31.30,0.0,NIL,0.0,31.30,31.30,31.30


## 2. Case attributes

In [6]:
# Configure the number of attributes to be included
INCLUDE_DATA = True
INCLUDE_TIME = True
INCLUDE_DELAYS = True

In [7]:
# Create a case attribute from the value of that variable at the final event in a case
TAKE_AT_FINAL = added_col_names

case_index = {}
activity_list = []
log_starting_time = df.at[0, TIMESTAMP_COL]
print('log starting time:', log_starting_time)
columns = df.columns.to_list()
for col in [CASE_ID_COL, ACTIVITY_COL, TIMESTAMP_COL]:
    columns.remove(col)
for index, row in df.iterrows():
    case_id = row[CASE_ID_COL]
    activity_name = row[ACTIVITY_COL]
    if not activity_name in activity_list:
        activity_list.append(activity_name)
    qualifier = activity_name + '.'
    if not case_id in case_index:
        case_record = {'case_id': case_id, qualifier+'count': 1, 'event_count': 1}
        if INCLUDE_TIME:
            case_record['start_time'] = row[TIMESTAMP_COL]
            case_record['start_time_rel'] = int((row[TIMESTAMP_COL]-log_starting_time).components.days)
            case_record['duration'] = 0
            case_record[qualifier+'start'] = 0
        for col in columns:
            if not pd.isna(row[col]):
                if INCLUDE_DATA:
                    case_record[qualifier+col] = row[col]
                if col in TAKE_AT_FINAL:
                    field_name = 'Final.' + col
                    case_record[field_name] = row[col]
        case_index[case_id] = case_record
    else:
        case_record = case_index[case_id]
        case_record['event_count'] += 1
        if qualifier+'start' in case_record:
            # print('Activity {} occurs multiple times in case {}.'.format(activity_name, case_id))
            activity_count = case_record[qualifier+'count'] + 1
            case_record[qualifier+'count'] = activity_count
            qualifier = activity_name + ':' + str(activity_count) + '.'
        else:
            case_record[qualifier+'count'] = 1

        if INCLUDE_TIME:
            duration = row[TIMESTAMP_COL] - case_record['start_time']
            case_record[qualifier+'start'] = int(duration.components.days)  # str()
            case_record['duration'] = int(duration.components.days)  # duration
        for col in columns:
            if not pd.isna(row[col]):
                if INCLUDE_DATA:
                    case_record[qualifier+col] = row[col]
                if col in TAKE_AT_FINAL:
                    field_name = 'Final.' + col
                    case_record[field_name] = row[col]

out_df = pd.DataFrame([case_index[case_id] for case_id in case_index])

# Fill nan counts with 0
for col in out_df.columns:
    if col.endswith('.count'):
        out_df[col] = out_df[col].fillna(0)

# Add delay for each pair of distinct activities
if INCLUDE_DELAYS:
    first_activities = activity_list.copy()
    while len(first_activities) > 0:
        first_act = first_activities.pop()
        second_activities = first_activities.copy()
        while len(second_activities) > 0:
            second_act = second_activities.pop()
            column_name = first_act + ':' + second_act + '.delay'
            # print('adding', column_name)
            out_df[column_name] = out_df[second_act+'.start'] - out_df[first_act+'.start']

out_df.info(verbose=True)

log starting time: 2000-01-01 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150370 entries, 0 to 150369
Data columns (total 374 columns):
 #    Column                                                                        Dtype         
---   ------                                                                        -----         
 0    case_id                                                                       object        
 1    Create Fine.count                                                             int64         
 2    event_count                                                                   int64         
 3    start_time                                                                    datetime64[ns]
 4    start_time_rel                                                                int64         
 5    duration                                                                      int64         
 6    Create Fine.start                                  

## 3. Case predicates from earlier analysis

In [8]:
# Add case predicates based on the existing case attributes, domain-specific through earlier analysis, e.g. in an outcome flow diagram
out_df['paid_full'] = out_df['Final.outstanding_balance'] <= 0.0
out_df['dismissed'] = out_df['Final.dismissal::last'].isin(['G', '#'])
out_df['credit_collection'] = out_df['Send for Credit Collection.count'] >= 1
out_df['unresolved'] = ~out_df['paid_full'] & ~out_df['credit_collection'] & ~out_df['dismissed']

# out_df['verify_0'] = (out_df['Final.totalPaymentAmount::max'] == out_df['Final.paymentAmount::sum'])
# out_df['verify_1'] = (out_df['Insert Fine Notification.count'] == out_df['Add penalty.count'])

out_df['appeal'] = (out_df['Appeal to Judge.count'] >= 1) | (out_df['Insert Date Appeal to Prefecture.count'] >= 1)
out_df['overturned_judge'] = (out_df['Appeal to Judge.count'] >= 1) & (out_df['Final.dismissal::last'] == 'G')
out_df['overturned_prefecture'] = (out_df['Insert Date Appeal to Prefecture.count'] >= 1) & (out_df['Final.dismissal::last'] == '#')
out_df['overturned'] = out_df['overturned_judge'] | out_df['overturned_prefecture']
out_df['upheld'] = out_df['appeal'] & ~out_df['overturned']

out_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150370 entries, 0 to 150369
Data columns (total 383 columns):
 #    Column                                                                        Dtype         
---   ------                                                                        -----         
 0    case_id                                                                       object        
 1    Create Fine.count                                                             int64         
 2    event_count                                                                   int64         
 3    start_time                                                                    datetime64[ns]
 4    start_time_rel                                                                int64         
 5    duration                                                                      int64         
 6    Create Fine.start                                                             int64       

In [9]:
output_file_name = 'cases_' + INPUT_FILE_NAME
out_df.to_csv(output_file_name, index=False)
print(len(out_df), 'cases written to', output_file_name)

150370 cases written to cases_rtfm_full.csv


## 4. Verification (optional)

In [10]:
# uncomment below for verification 0

# print(out_df['verify_0'].value_counts())
# v_df = out_df[out_df['verify_0'] == False]
# v_df[['case_id', 'Final.totalPaymentAmount::max', 'Final.paymentAmount::sum']].head()

In [11]:
# uncomment below for verification 1

# print(out_df['verify_1'].value_counts())
